# Product SVD in Python


In this NoteBook the reader finds code to read a GeoTiff file, single- or multi-band, from HDFS. It reads the GeoTiff as a **ByteArray** and then stores the GeoTiff in memory using **MemFile** from **RasterIO** python package. Subsequently, the Python module _productsvd_ is used to determine the SVD of two phenology datasets.

## Dependencies

In [ ]:
#Add all dependencies to PYTHON_PATH
import sys
sys.path.append("/usr/lib/spark/python")
sys.path.append("/usr/lib/spark/python/lib/py4j-0.10.4-src.zip")
sys.path.append("/usr/lib/python3/dist-packages")
sys.path.append("/data/local/jupyterhub/modules/python")

#Define environment variables
import os
os.environ["HADOOP_CONF_DIR"] = "/etc/hadoop/conf"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "ipython"

import subprocess

#Load PySpark to connect to a Spark cluster
from pyspark import SparkConf, SparkContext
from hdfs import InsecureClient
from tempfile import TemporaryFile

#from osgeo import gdal
#To read GeoTiffs as a ByteArray
from io import BytesIO
from rasterio.io import MemoryFile

import numpy as np
import pandas
import datetime
import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from os import listdir
from os.path import isfile, join
from numpy import exp, log
from numpy.random import standard_normal
from scipy.linalg import norm, qr, svd
from productsvd import qrproductsvd
from sklearn.utils.extmath import randomized_svd

## Configuration

In [ ]:
debugMode = True

## Connect to Spark

In [ ]:
appName = "plot_GeoTiff"
masterURL = "spark://pheno0.phenovari-utwente.surf-hosted.nl:7077"

#A context needs to be created if it does not already exist
try:
    sc.stop()
except NameError:
    print("A new Spark Context will be created.")

sc = SparkContext(conf = SparkConf().setAppName(appName).setMaster(masterURL))
conf = sc.getConf()

## Support functions

In [ ]:
def dprint(msg):
    if (debugMode):
        print(str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + " | " + msg)

In [ ]:
def progressBar(message, value, endvalue, bar_length = 20):
    if (debugMode):
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write("\r" 
                         + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
                         + " | " 
                         + message 
                         + ": [{0}] {1}%".format(arrow + spaces, int(round(percent * 100)))
                        )
        if value == endvalue:
            sys.stdout.write("\n")
        sys.stdout.flush()

In [ ]:
def get_hdfs_client():
    return InsecureClient("pheno0.phenovari-utwente.surf-hosted.nl:50070", user="pheno",
         root="/")

In [ ]:
def getDataSet(directoryPath):
    dprint("Running getDataSet(directoryPath)")
    
    files = sc.binaryFiles(directoryPath + "/*.tif")
    fileList = files.keys().collect()
    dprint("Number of files: " + str(len(fileList)))
    dataSet = []
    plotShapes = []
    flattenedShapes = []
    for i, f in enumerate(fileList):
        progressBar("Reading files", i + 1, len(fileList))
        data = files.lookup(f)
        dataByteArray = bytearray(data[0])
        memfile = MemoryFile(dataByteArray)
        dataset = memfile.open()
        relevantBand = np.array(dataset.read()[0])
        memfile.close()
        plotShapes.append(relevantBand.shape)
        flattenedDataSet = relevantBand.flatten()
        flattenedShapes.append(flattenedDataSet.shape)
        dataSet.append(flattenedDataSet)
    dataSet = np.array(dataSet).T
    dprint("dataSet.shape: " + str(dataSet.shape))
    
    dprint("Ending getDataSet(directoryPath)")
    return dataSet

In [ ]:
def getMask(filePath):
    dprint("Running getMask(filePath)")
    
    mask_data = sc.binaryFiles(filePath).take(1)
    mask_byteArray = bytearray(mask_data[0][1])
    mask_memfile = MemoryFile(mask_byteArray)
    mask_dataset = mask_memfile.open()
    mask_data = np.array(mask_dataset.read()[0])
    mask_memfile.close()
    dprint("mask_data.shape: " + str(mask_data.shape))
    
    dprint("Ending getMask(filePath)")
    return mask_data

In [ ]:
def filterDataSet(dataSet, maskData):
    dprint("Running filterDataSet(dataSet, maskIndex)")
    
    maskIndex = np.nonzero(maskData.flatten())[0]
    dataSetFiltered = dataSet[maskIndex]
    dprint("dataSetFiltered.shape: " + str(dataSetFiltered.shape))
    
    dprint("Ending filterDataSet(dataSet, maskIndex)")
    return dataSetFiltered

In [ ]:
def validateNorms(dataSet1, dataSet2, U, s, V):
    dprint("Running validateNorms(dataSet1, dataSet2, U, s, V)")
    
    length = len(s)
    norms = []
    for i in range(length):
        progressBar("Validating norms", i + 1, length)
        u = dataSet1 @ (dataSet2.T @ V.T[i]) / s[i]
        v = dataSet2 @ (dataSet1.T @ U.T[i]) / s[i]
        norms.append(norm(U.T[i] - u))
        norms.append(norm(V.T[i] - v))
    dprint("Largest norm difference: " + str(max(norms)))
    
    dprint("Ending validateNorms(dataSet1, dataSet2, U, s, V)")

In [ ]:
def writeMode(resultDirectory, fileName, i, U, s, V):
    dprint("Running writeMode(resultDirectory, fileName, i, U, s, V)")
    
    inFile = "/tmp/" + fileName
    outFile = resultDirectory + fileName
    
    decompositionFile = open(inFile, "w")
    U.T[i].tofile(decompositionFile, sep = ",")
    decompositionFile.close()
    decompositionFile = open(inFile, "a")
    decompositionFile.write("\n")
    s[i].tofile(decompositionFile, sep = ",")
    decompositionFile.write("\n")
    V.T[i].tofile(decompositionFile, sep = ",")
    decompositionFile.close()
    
    #Upload to HDFS
    subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  

    #Remove from /tmp/
    subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending writeMode(resultDirectory, fileName, i, U, s, V)")

In [ ]:
def writeCSV(resultDirectory, fileName, res):
    dprint("Running writeCSV(resultDirectory, fileName, res)")
    
    inFile = "/tmp/" + fileName
    outFile = resultDirectory + fileName
    
    decompositionFile = open(inFile, "w")
    res.T.tofile(decompositionFile, sep = ",")
    decompositionFile.close()
    
    #Upload to HDFS
    subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  

    #Remove from /tmp/
    subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending writeCSV(resultDirectory, fileName, res)")

In [ ]:
def plotMode(resultDirectory, fileName, singularVector, maskData):
    dprint("Running plotMode(resultDirectory, fileName, singularVector, maskData)")
    
    inFile = "/tmp/" + fileName
    outFile = resultDirectory + fileName
    
    data = np.zeros(maskData.shape[0] * maskData.shape[1])
    maskIndex = np.nonzero(maskData.flatten())[0]
    np.put(data, maskIndex, singularVector)
    data = np.reshape(data, maskData.shape)
    plt.figure(1)
    cmap = plt.cm.get_cmap('YlGn')
    img = plt.imshow(data.T, cmap = 'YlGn')
    plt.colorbar(orientation = 'horizontal')
    plt.clim(float(np.min(data.T)), float(np.max(data.T)))
    plt.axis('off')
    plt.savefig(inFile)
    plt.clf()

    #Upload to HDFS
    subprocess.run(['hadoop', 'dfs', '-copyFromLocal', '-f', inFile, outFile])  

    #Remove from /tmp/
    subprocess.run(['rm', '-fr', inFile])
    
    dprint("Ending plotMode(resultDirectory, fileName, singularVector, shape)")

In [ ]:
def runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory):
    dprint("Running runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)")

    dataSet1 = getDataSet(dataDirectory1)
    dataSet2 = getDataSet(dataDirectory2)
    
    maskData = getMask(maskFile)
    
    dataSetFiltered1 = filterDataSet(dataSet1, maskData)
    dataSetFiltered2 = filterDataSet(dataSet2, maskData)
    
    U, s, Vt = qrproductsvd(dataSetFiltered1, dataSetFiltered2)
    V = Vt.T
    dprint("U.shape: " + str(U.shape))
    dprint("s.shape: " + str(s.shape))
    dprint("V.shape: " + str(V.shape))
    dprint("Singular values of product: ")
    dprint(str(s))
    
    validateNorms(dataSetFiltered1, dataSetFiltered2, U, s, V)
    
    for i in range(len(s)):
        progressBar("Writing modes", i + 1, len(s))
        iString = str(i + 1).zfill(2)
        writeMode(resultDirectory, "Mode" + iString + ".txt", i, U, s, V)
        plotMode(resultDirectory, "ModeU" + iString + ".pdf", U.T[i], maskData)
        plotMode(resultDirectory, "ModeV" + iString + ".pdf", V.T[i], maskData)
    writeCSV(resultDirectory, "U.csv", U)
    writeCSV(resultDirectory, "s.csv", s)
    writeCSV(resultDirectory, "V.csv", V)
    
    dprint("Ending runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)")

## Analyses

### Analysis 1

In [ ]:
dprint("-------------------------------")
dprint("Running analysis 1")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomGridmet/"
dataDirectory2 = "hdfs:///user/hadoop/spring-index/LeafGridmet/"
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/pheno/svd/BloomGridmetLeafGridmet/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 1")
dprint("-------------------------------")

### Analysis 2

In [ ]:
dprint("-------------------------------")
dprint("Running analysis 2")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/BloomGridmet/"
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOST4Km/"
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/pheno/svd/BloomGridmetSOST4Km/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 2")
dprint("-------------------------------")

### Analysis 3

In [ ]:
dprint("-------------------------------")
dprint("Running analysis 3")
dprint("-------------------------------")

dataDirectory1 = "hdfs:///user/hadoop/spring-index/LeafGridmet/"
dataDirectory2 = "hdfs:///user/hadoop/avhrr/SOST4Km/"
maskFile = "hdfs:///user/hadoop/usa_mask_gridmet.tif"
resultDirectory = "hdfs:///user/pheno/svd/LeafGridmetSOST4Km/"

#Create Result dir
subprocess.run(['hadoop', 'dfs', '-mkdir', resultDirectory])

runAnalysis(dataDirectory1, dataDirectory2, maskFile, resultDirectory)

dprint("-------------------------------")
dprint("Ending analysis 3")
dprint("-------------------------------")

End of Notebook